In [1]:
import os
import random
import numpy as np
import torch
import config

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-02-15 19:04:05 - get_logger - Writing log to logs/graphnet_20230215-190405.log


In [2]:
def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=cfg.WARM_UP_PCT * cfg.EPOCHS,
        num_training_steps=cfg.EPOCHS,
    )



    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )

In [3]:
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

In [4]:
main('BASELINE_graph_V5')

Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_30', 'LOSS_FUNC': <class 'icecube.modelsgraph.gVonMisesFisher3DLossCosineSimularityLoss'>, 'NUM_WORKERS': 22, 'MODEL_NAME': <class 'icecube.modelsgraph.EGNNModelV4'>, 'FIT_FUNC': <function gfit_shuflle at 0x7f675f6b6950>, 'METRIC': <function gget_score_vector at 0x7f675f6b6cb0>, 'TRN_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'VAL_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'TRN_BATCH_RANGE': [[1, 100], [100, 200], [200, 300], [300, 400], [400, 500], [500, 600]], 'EPOCHS': 12, '__doc__': None}


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [1, 100]


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,


...

Better model found at epoch 9 with value: 1.0507103204727173.
   epoch  train_loss  valid_loss     metric
0      9    1.111517    1.109453  1.0507103
trn_range: [400, 500]


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 10 with value: 1.0441936254501343.
   epoch  train_loss  valid_loss     metric
0     10    1.108522    1.106271  1.0441936
trn_range: [500, 600]


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,


Better model found at epoch 11 with value: 1.0379496812820435.
   epoch  train_loss  valid_loss     metric
0     11    1.104324    1.096044  1.0379497
Training done


In [5]:
k

NameError: name 'k' is not defined

In [6]:
k

NameError: name 'k' is not defined